In [16]:
import numpy as np
import cupy as cp
import time
import requests
import io
from typing import Tuple, Optional
import matplotlib.pyplot as plt

In [17]:
class GPUSimulatedAnnealing:
    """GPU-accelerated Simulated Annealing for QUBO problems"""

    def __init__(self, Q: cp.ndarray, n_runs: int = 1024, device_id: int = 0):
        """
        Initialize GPU SA solver

        Args:
            Q: QUBO matrix (CuPy array)
            n_runs: Number of parallel SA runs
            device_id: GPU device ID
        """
        cp.cuda.Device(device_id).use()
        self.Q = Q
        self.n = Q.shape[0]
        self.n_runs = n_runs

        # Initialize random states for each run
        self.states = cp.random.randint(0, 2, (n_runs, self.n), dtype=cp.int8)
        self.best_states = self.states.copy()

        # Calculate initial energies
        self.current_energies = self._calculate_energies(self.states)
        self.best_energies = self.current_energies.copy()
        self.global_best_energy = cp.max(self.best_energies)
        self.global_best_state = self.best_states[cp.argmax(self.best_energies)].copy()

    def _calculate_energies(self, states: cp.ndarray) -> cp.ndarray:
        """Calculate QUBO energies for all states"""
        # Energy = x^T Q x for each state
        energies = cp.sum(states[:, :, None] * self.Q[None, :, :] * states[:, None, :], axis=(1, 2))
        return energies

    def _calculate_energy_single(self, state: cp.ndarray) -> float:
        """Calculate energy for a single state"""
        return cp.sum(state[:, None] * self.Q * state[None, :])

    def solve(self, max_iter: int = 10000, initial_temp: float = 100.0,
              cooling_rate: float = 0.95, min_temp: float = 0.01) -> Tuple[cp.ndarray, float]:
        """
        Run parallel simulated annealing

        Args:
            max_iter: Maximum iterations per temperature
            initial_temp: Initial temperature
            cooling_rate: Temperature cooling rate
            min_temp: Minimum temperature

        Returns:
            best_state: Best solution found
            best_energy: Best energy (objective value)
        """
        temperature = initial_temp
        iteration = 0

        print(f"Starting GPU SA with {self.n_runs} parallel runs...")
        start_time = time.time()

        while temperature > min_temp and iteration < max_iter:
            # Generate random bit flips for all runs
            flip_positions = cp.random.randint(0, self.n, size=self.n_runs)

            # Create new states by flipping one bit
            new_states = self.states.copy()
            new_states[cp.arange(self.n_runs), flip_positions] = 1 - new_states[cp.arange(self.n_runs), flip_positions]

            # Calculate new energies
            new_energies = self._calculate_energies(new_states)

            # Calculate energy differences
            energy_diffs = new_energies - self.current_energies

            # Accept/reject based on Metropolis criterion
            accept_probs = cp.where(energy_diffs > 0, 1.0, cp.exp(energy_diffs / temperature))
            random_vals = cp.random.random(self.n_runs)
            accept_mask = random_vals < accept_probs

            # Update current states and energies
            self.states[accept_mask] = new_states[accept_mask]
            self.current_energies[accept_mask] = new_energies[accept_mask]

            # Update best states
            better_mask = self.current_energies > self.best_energies
            self.best_states[better_mask] = self.states[better_mask]
            self.best_energies[better_mask] = self.current_energies[better_mask]

            # Update global best
            current_global_best = cp.max(self.best_energies)
            if current_global_best > self.global_best_energy:
                self.global_best_energy = current_global_best
                self.global_best_state = self.best_states[cp.argmax(self.best_energies)].copy()

            # Cool down
            if iteration % 100 == 0:
                temperature *= cooling_rate
                if iteration % 1000 == 0:
                    print(f"Iteration {iteration}, Temp: {temperature:.6f}, Best: {float(self.global_best_energy):.2f}")

            iteration += 1

        end_time = time.time()
        print(f"GPU SA completed in {end_time - start_time:.2f} seconds")
        print(f"Best energy found: {float(self.global_best_energy):.2f}")

        return self.global_best_state, float(self.global_best_energy)

class MaxCutQUBO:
    """Max Cut problem solver using QUBO formulation"""

    def __init__(self, adjacency_matrix: np.ndarray):
        """
        Initialize Max Cut QUBO

        Args:
            adjacency_matrix: Graph adjacency matrix
        """
        self.adj_matrix = adjacency_matrix
        self.n = adjacency_matrix.shape[0]
        self.Q = self._build_qubo_matrix()

    def _build_qubo_matrix(self) -> np.ndarray:
        """
        Build QUBO matrix for Max Cut problem

        For Max Cut, we want to maximize: sum_{i,j} w_{ij} * x_i * (1-x_j) + w_{ij} * (1-x_i) * x_j
        This simplifies to: sum_{i,j} w_{ij} * (x_i + x_j - 2*x_i*x_j)

        QUBO form: x^T Q x where Q_{ij} = -w_{ij} for i≠j, Q_{ii} = sum_j w_{ij}
        """
        Q = np.zeros((self.n, self.n))

        # Off-diagonal elements: -w_{ij}
        Q = -self.adj_matrix.copy()

        # Diagonal elements: sum of weights for each node
        for i in range(self.n):
            Q[i, i] = np.sum(self.adj_matrix[i, :])

        return Q

    def evaluate_cut(self, solution: np.ndarray) -> int:
        """Evaluate the cut value for a given solution"""
        cut_value = 0
        for i in range(self.n):
            for j in range(i+1, self.n):
                if solution[i] != solution[j]:  # Different partitions
                    cut_value += self.adj_matrix[i, j]
        return cut_value

# Best known solutions for G-Set instances (from literature)
GSET_BEST_KNOWN = {
    'G1': 11624,   'G2': 11620,   'G3': 11622,   'G4': 11646,   'G5': 11631,
    'G6': 2178,    'G7': 2006,    'G8': 2005,    'G9': 2054,    'G10': 2000,
    'G11': 564,    'G12': 556,    'G13': 582,    'G14': 3064,   'G15': 3050,
    'G16': 3052,   'G17': 3047,   'G18': 992,    'G19': 906,    'G20': 941,
    'G21': 931,    'G22': 13359,  'G23': 13344,  'G24': 13337,  'G25': 13340,
    'G26': 13328,  'G27': 3341,   'G28': 3298,   'G29': 3405,   'G30': 3403,
    'G31': 3288,   'G32': 1410,   'G33': 1382,   'G34': 1384,   'G35': 7687,
    'G36': 7680,   'G37': 7691,   'G38': 7688,   'G39': 2408,   'G40': 2400,
    'G41': 2405,   'G42': 2481,   'G43': 6660,   'G44': 6650,   'G45': 6654,
    'G46': 6649,   'G47': 6657,   'G48': 6000,   'G49': 6000,   'G50': 5880,
    'G51': 4054,   'G52': 4049,   'G53': 4013,   'G54': 4030,   'G55': 10299,
    'G56': 4006,   'G57': 3494,   'G58': 3413,   'G59': 3481,   'G60': 3494,
    'G61': 3493,   'G62': 3480,   'G63': 3297,   'G64': 3318,   'G65': 4026,
    'G66': 6358,   'G67': 6363,   'G70': 9541,   'G72': 6714,   'G77': 6723,
    'G81': 9926
}

def get_best_known_solution(instance_name: str) -> int:
    """Get the best known solution for a G-Set instance"""
    return GSET_BEST_KNOWN.get(instance_name, None)

def calculate_solution_quality(found_value: int, best_known: int) -> dict:
    """Calculate solution quality metrics"""
    if best_known is None:
        return {'gap': None, 'ratio': None, 'quality': 'Unknown'}

    gap = best_known - found_value
    ratio = found_value / best_known
    percentage = ratio * 100

    return {
        'gap': gap,
        'ratio': ratio,
        'percentage': percentage,
        'quality': f"{percentage:.2f}%"
    }

def load_gset_instance(instance_name: str) -> np.ndarray:
    """
    Load a G-Set instance from the web

    Args:
        instance_name: Name of the instance (e.g., 'G1', 'G2', etc.)

    Returns:
        adjacency_matrix: Graph adjacency matrix
    """
    url = f"https://web.stanford.edu/~yyye/yyye/Gset/{instance_name}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        lines = response.text.strip().split('\n')

        # First line contains n (nodes) and m (edges)
        n, m = map(int, lines[0].split())

        # Initialize adjacency matrix
        adj_matrix = np.zeros((n, n), dtype=np.float32)

        # Parse edges
        for i in range(1, min(m+1, len(lines))):
            parts = lines[i].split()
            if len(parts) >= 3:
                u, v, w = int(parts[0])-1, int(parts[1])-1, float(parts[2])  # Convert to 0-based indexing
                adj_matrix[u, v] = w
                adj_matrix[v, u] = w  # Undirected graph

        print(f"Loaded {instance_name}: {n} nodes, {m} edges")
        return adj_matrix

    except Exception as e:
        print(f"Error loading {instance_name}: {e}")
        # Return a small random graph for testing
        print("Generating random test graph instead...")
        np.random.seed(42)
        n = 20
        adj_matrix = np.random.randint(0, 2, (n, n)).astype(np.float32)
        adj_matrix = (adj_matrix + adj_matrix.T) / 2  # Make symmetric
        np.fill_diagonal(adj_matrix, 0)  # No self-loops
        return adj_matrix

def benchmark_gpu_sa():
    """Benchmark GPU SA on G-Set instances"""

    # Check if CuPy is available
    try:
        cp.cuda.Device(0).use()
        print("GPU detected and ready!")
    except Exception as e:
        print(f"GPU not available: {e}")
        return

    # Test instances (start with smaller ones)
    instances = ['G1', 'G2', 'G3', 'G4']  # Mix of different sizes for testing

    results = []

    for instance_name in instances:
        print(f"\n{'='*50}")
        print(f"Benchmarking {instance_name}")
        print(f"{'='*50}")

        # Load instance
        adj_matrix = load_gset_instance(instance_name)

        # Create QUBO problem
        maxcut_qubo = MaxCutQUBO(adj_matrix)

        # Convert to GPU
        Q_gpu = cp.asarray(maxcut_qubo.Q)

        # Solve with GPU SA
        gpu_sa = GPUSimulatedAnnealing(Q_gpu, n_runs=800)  # Use fewer runs for larger instances

        start_time = time.time()
        best_solution_gpu, best_energy_gpu = gpu_sa.solve(
            max_iter=6000,
            initial_temp=8.0,
            cooling_rate=0.95,
            min_temp=0.01
        )
        gpu_time = time.time() - start_time

        # Convert back to CPU for evaluation
        best_solution_cpu = cp.asnumpy(best_solution_gpu)

        # Evaluate the cut
        cut_value = maxcut_qubo.evaluate_cut(best_solution_cpu)

        # Compare with best known solution
        best_known = get_best_known_solution(instance_name)
        quality_metrics = calculate_solution_quality(cut_value, best_known)

        print(f"\nResults for {instance_name}:")
        print(f"Cut value found: {cut_value}")
        if best_known is not None:
            print(f"Best known value: {best_known}")
            print(f"Gap from optimal: {quality_metrics['gap']} ({quality_metrics['quality']})")
            if quality_metrics['gap'] == 0:
                print("🎉 OPTIMAL SOLUTION FOUND!")
            elif quality_metrics['percentage'] >= 99.0:
                print("🔥 Excellent solution (≥99% of optimal)")
            elif quality_metrics['percentage'] >= 95.0:
                print("✅ Very good solution (≥95% of optimal)")
            elif quality_metrics['percentage'] >= 90.0:
                print("👍 Good solution (≥90% of optimal)")
            else:
                print("📈 Room for improvement")
        else:
            print("Best known value: Unknown")
        print(f"QUBO energy: {best_energy_gpu:.2f}")
        print(f"GPU time: {gpu_time:.2f} seconds")
        print(f"Solution: {best_solution_cpu[:min(20, len(best_solution_cpu))]}...")  # Show first 20 bits

        results.append({
            'instance': instance_name,
            'nodes': adj_matrix.shape[0],
            'cut_value': cut_value,
            'best_known': best_known,
            'gap': quality_metrics['gap'],
            'quality_percentage': quality_metrics.get('percentage', None),
            'qubo_energy': best_energy_gpu,
            'gpu_time': gpu_time
        })

    # Summary
    print(f"\n{'='*70}")
    print("BENCHMARK SUMMARY")
    print(f"{'='*70}")
    print(f"{'Instance':<10} {'Nodes':<8} {'Found':<8} {'Best':<8} {'Gap':<6} {'Quality':<10} {'Time(s)':<8}")
    print("-" * 70)

    total_gap = 0
    total_instances = 0

    for result in results:
        instance = result['instance']
        nodes = result['nodes']
        found = result['cut_value']
        best = result['best_known'] if result['best_known'] else 'N/A'
        gap = result['gap'] if result['gap'] is not None else 'N/A'
        quality = f"{result['quality_percentage']:.1f}%" if result['quality_percentage'] else 'N/A'
        time_taken = f"{result['gpu_time']:.2f}"

        print(f"{instance:<10} {nodes:<8} {found:<8} {best:<8} {gap:<6} {quality:<10} {time_taken:<8}")

        if result['gap'] is not None:
            total_gap += result['gap']
            total_instances += 1

    if total_instances > 0:
        avg_gap = total_gap / total_instances
        print("-" * 70)
        print(f"Average gap from optimal: {avg_gap:.1f}")

        # Calculate overall performance
        solved_optimally = sum(1 for r in results if r['gap'] == 0)
        near_optimal = sum(1 for r in results if r['quality_percentage'] and r['quality_percentage'] >= 99.0)
        good_solutions = sum(1 for r in results if r['quality_percentage'] and r['quality_percentage'] >= 95.0)

        print(f"Optimal solutions: {solved_optimally}/{total_instances}")
        print(f"Near-optimal (≥99%): {near_optimal}/{total_instances}")
        print(f"Good solutions (≥95%): {good_solutions}/{total_instances}")

    return results

if __name__ == "__main__":
    # Run the benchmark
    benchmark_results = benchmark_gpu_sa()

GPU detected and ready!

Benchmarking G14
Loaded G14: 800 nodes, 4694 edges
Starting GPU SA with 800 parallel runs...
Iteration 0, Temp: 7.600000, Best: 2448.00


KeyboardInterrupt: 